Le assunzioni che faccio è che la prima stringa sia fatta dalla generale
la seconda stringa indica come si calcola il primo elemento e la terza indica come si calcola il secondo elemento e cosi via e poi se gli elementi coinvolti sono coinvolti nel primo allora si fa quello. la cosa deve essere ricorsiva. Quindi parsata la stringa devo andare a controllare la stringa dopo e cosi via. Possiamo fare una logica ricorsiva

In [8]:
#proviamo il parsing delle stringhe
import re
from collections import defaultdict
stringa="A°mean°mo[ S°/[ R°consumption_sum°T°m°o° ; R°time_sum°T°m°o° ]]"
stringa1="S°+[ R°time_sum°T°M°idle° ; R°time_sum°T°M°offline° ]"
stringa3="A°sum°mo[ A°sum°t[ D°consumption_sum°t°m°o° ] ]"

#ho bisogno di isolare i nomi dei kpi le operazioni, tempo e machine li abbiamo dal RAG
#ho bidogno di altre cose da fare tipo andare a identificare le lettere tipo A e B

#trovare la logica migliore per il parsing di queste 3 stringhe
#pattern = r"([A-Z])°([\w]+)°([\w]+)\[|\[|\]|;|([A-Z])°([\w]+)°([\w]+)"

#patter per dividere tutto ciò che trova nella stringa, ma come si fa a dare significato ad ogni punto?
#token_pattern = r"[A-Z]+|[a-zA-Z0-9_]+|[\[\]/;+\-*/=]"

#questo cerco di dividere solo quello dentro le parentesi quadre funziona?? può essere utile
pattern = r"([A-Z])°([a-zA-Z0-9__+\-*/=]+)"

#posso fare questa assunzione? sono mappate uno a uno con le macchine
pattern_oper = r"T°M°([a-zA-Z0-9_]+)"

#altro possibile pattern
pattern_pp = r"([a-zA-Z0-9_]+)°([a-zA-Z0-9_]+)(?:°([a-zA-Z0-9_]+))?"

#find all ritorna una lista che poi è da parsare
a=set(re.findall(r"\b[A-Za-z][A-Za-z0-9]*\b", stringa))

#posso affrontare il problema in diversi modi
b=re.findall(pattern, stringa)
#faccio il parsing delle stringhe e vediamo

'''
dizionario=defaultdict(list)

for key, value in b:
    dizionario[key].append(value)

print(dict(dizionario))
'''
b

[('A', 'mean'),
 ('S', '/'),
 ('R', 'consumption_sum'),
 ('T', 'm'),
 ('R', 'time_sum'),
 ('T', 'm')]

Abbiamo ottenuto una lista per ogni lettera e delle variabili e les tringhe sono parsate. Adesso devo stabilire una logica

In [ ]:
import re
from collections import defaultdict
from datetime import datetime

#Mi arrivano queste tre stringhe
stringa="A°mean°mo[ S°/[ R°consumption_sum°T°m°o° ; R°time_sum°T°m°o° ]]"
stringa1="S°+[ R°time_sum°T°M°idle° ; R°time_sum°T°M°offline° ]"
stringa3="A°sum°mo[ A°sum°t[ D°consumption_sum°t°m°o° ] ]"

# a me arriva questo che è la formula
formule=["A°mean°mo[ S°/[ R°consumption_sum°T°m°o° ; R°time_sum°T°m°o° ]]",
         "A°sum°mo[ A°sum°t[ D°consumption_sum°t°m°o° ] ]",
         "S°+[ R°time_sum°T°M°idle° ; R°time_sum°T°M°offline° ]"]

#io ho già informazioni del tipo

step=7
macchine=['ma','mb']
operazioni=['idle','offline']
start_date=datetime(2023, 1, 1),
end_date=datetime(2023, 12, 31),
aggregazione='mean'

############################

#Logica di parsing delle stringhe
pattern1 = r"([a-zA-Z0-9_/\+\-\*\=]+(?:°[a-zA-Z0-9_/\+\-\*\=]+)*)"
# Troviamo tutti i match principali
matches = re.findall(pattern1, stringa)
# Dividiamo ogni match in tuple
parsed_results = [tuple(item.split("°")) for item in matches]
# Risultato finale
print(parsed_results)

[('A', 'mean', 'mo'), ('S', '/'), ('R', 'consumption_sum', 'T', 'm', 'o'), ('R', 'time_sum', 'T', 'm', 'o')]
